# Análise do Efeito da Temperatura na Destilação de Conhecimento - Parte 1

Este notebook investiga como o parâmetro de temperatura afeta o processo de destilação de conhecimento usando a biblioteca DeepBridge. Exploraremos tanto os aspectos teóricos quanto práticos da temperatura na destilação, com visualizações e experimentos sistemáticos.

## 1. Introdução à Temperatura na Destilação

### O que é a Temperatura na Destilação de Conhecimento?

Na destilação de conhecimento, a "temperatura" (T) é um hiperparâmetro que controla a suavidade das distribuições de probabilidade produzidas pelo modelo professor antes de serem usadas para treinar o modelo aluno.

Matematicamente, a temperatura é aplicada às logits (valores pré-softmax) do modelo professor da seguinte forma:

$$q_i = \frac{\exp(z_i/T)}{\sum_j \exp(z_j/T)}$$

Onde:
- $z_i$ são as logits originais do modelo professor
- $T$ é o parâmetro de temperatura
- $q_i$ são as probabilidades suavizadas

#### Importância da Temperatura

- **T = 1**: Comportamento padrão (softmax normal)
- **T > 1**: Produz distribuições mais suaves, revelando mais informação sobre as relações entre classes
- **T < 1**: Produz distribuições mais acentuadas, aproximando-se de one-hot encoding

O efeito da temperatura é crucial porque determina a quantidade de "conhecimento escuro" transferido do professor para o aluno.

## 2. Configuração do Ambiente e Importação de Bibliotecas

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from scipy.special import softmax
import warnings

# Importações da biblioteca DeepBridge
from deepbridge.db_data import DBDataset
from deepbridge.auto_distiller import AutoDistiller
from deepbridge.distillation.classification.model_registry import ModelType
from deepbridge.auto.config import DistillationConfig
from deepbridge.visualizer.distribution_visualizer import DistributionVisualizer

# Configurar o estilo das visualizações
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style="whitegrid")
warnings.filterwarnings('ignore')

# Configurar o tamanho das figuras
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 3. Efeito Visual da Temperatura nas Distribuições

Para entender melhor o efeito da temperatura, vamos visualizar como diferentes valores de temperatura transformam as distribuições de probabilidade:

In [ ]:
def visualize_temperature_effect(logits, temperatures=[0.5, 1.0, 2.0, 5.0, 10.0]):
    """Visualiza o efeito da temperatura nas distribuições de probabilidade."""
    fig, axes = plt.subplots(len(temperatures), 1, figsize=(12, 4*len(temperatures)))
    
    # Certifique-se de que axes seja sempre uma lista, mesmo com um único subplot
    if len(temperatures) == 1:
        axes = [axes]
    
    for i, T in enumerate(temperatures):
        # Aplicar temperatura nas logits
        scaled_logits = logits / T
        probs = softmax(scaled_logits, axis=1)
        
        # Plotar a distribuição de probabilidades
        for j in range(min(5, probs.shape[0])):  # Limitar a 5 exemplos para clareza
            axes[i].bar(range(probs.shape[1]), probs[j], alpha=0.7, 
                      label=f'Exemplo {j+1}' if i == 0 else "")
            
        axes[i].set_title(f'Temperatura T = {T}')
        axes[i].set_xlabel('Classes')
        axes[i].set_ylabel('Probabilidade')
        axes[i].set_ylim(0, 1)
        
        # Calcular entropia para esta temperatura
        entropy = -np.sum(probs * np.log(probs + 1e-10), axis=1).mean()
        axes[i].text(0.02, 0.95, f'Entropia Média: {entropy:.4f}', 
                   transform=axes[i].transAxes, bbox=dict(facecolor='white', alpha=0.8))
        
    # Legenda apenas no primeiro gráfico
    axes[0].legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Gerar logits sintéticos para demonstração
np.random.seed(42)
synthetic_logits = np.random.randn(5, 10) * 2  # 5 exemplos, 10 classes

# Tornar a classe 3 tipicamente a mais forte para demonstração
synthetic_logits[:, 3] += 2

# Visualizar o efeito
visualize_temperature_effect(synthetic_logits)

### Análise do Efeito da Temperatura:

Como podemos observar nos gráficos acima:

1. **T = 0.5**: Com temperatura baixa, as probabilidades se concentram fortemente na classe dominante, aproximando-se de uma codificação one-hot. Isso resulta em menor entropia.

2. **T = 1.0**: Representa o comportamento padrão do softmax, onde a distribuição já mostra alguma incerteza entre classes.

3. **T = 2.0**: A distribuição se torna mais suave, revelando mais informação sobre as relações entre classes. A entropia aumenta.

4. **T = 5.0** e **T = 10.0**: Com temperaturas muito altas, as distribuições se aproximam de uma distribuição uniforme, onde as diferenças sutis entre classes são amplificadas. A entropia continua aumentando.

Este efeito de suavização é essencial para a destilação porque:
- Revela o "conhecimento escuro" do modelo professor
- Expõe relações entre classes que não são evidentes nas previsões binárias
- Permite que o modelo aluno aprenda padrões mais sutis

## 4. Preparação dos Dados para Experimentos

In [ ]:
# Criar um conjunto de dados sintético para experimentação
X, y = make_classification(
    n_samples=5000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    random_state=42
)

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar um modelo professor (Random Forest)
teacher_model = RandomForestClassifier(n_estimators=100, random_state=42)
teacher_model.fit(X_train, y_train)

# Gerar probabilidades do modelo professor
train_probs = teacher_model.predict_proba(X_train)
test_probs = teacher_model.predict_proba(X_test)

# Criar DataFrames para as probabilidades
train_prob_df = pd.DataFrame(train_probs, columns=['prob_class_0', 'prob_class_1'])
test_prob_df = pd.DataFrame(test_probs, columns=['prob_class_0', 'prob_class_1'])

# Criar DataFrames para os dados de entrada
train_df = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X_train.shape[1])])
train_df['target'] = y_train
train_df = pd.concat([train_df, train_prob_df], axis=1)

test_df = pd.DataFrame(X_test, columns=[f'feature_{i}' for i in range(X_test.shape[1])])
test_df['target'] = y_test
test_df = pd.concat([test_df, test_prob_df], axis=1)

# Criar um DBDataset
dataset = DBDataset(
    train_data=train_df,
    test_data=test_df,
    target_column='target',
    prob_cols=['prob_class_0', 'prob_class_1']
)

print(f"Conjunto de dados criado: {len(train_df)} amostras de treino, {len(test_df)} amostras de teste")
print(f"Performance do modelo professor no conjunto de teste:")
print(f"  Acurácia: {accuracy_score(y_test, teacher_model.predict(X_test)):.4f}")
print(f"  AUC-ROC: {roc_auc_score(y_test, test_probs[:, 1]):.4f}")
print(f"  Log Loss: {log_loss(y_test, test_probs):.4f}")

## 5. Experimentos com Diferentes Temperaturas

Agora, vamos explorar como diferentes valores de temperatura afetam o desempenho dos modelos destilados. Executaremos uma série de experimentos sistemáticos com diferentes combinações de modelos, temperaturas e valores de alpha (peso entre o erro de destilação e o erro de classificação).

In [ ]:
# Configurar experimentação sistemática com diferentes temperaturas
temperatures = [0.5, 1.0, 2.0, 3.0, 5.0, 10.0]
alphas = [0.3, 0.5, 0.7, 0.9]

# Modelos a serem testados
model_types = [
    ModelType.LOGISTIC_REGRESSION,
    ModelType.DECISION_TREE,
    ModelType.GBM,
]

# Diretório para salvar resultados
output_dir = "temperatura_destilacao_resultados"
os.makedirs(output_dir, exist_ok=True)

# Configuração do AutoDistiller
config = DistillationConfig(
    output_dir=output_dir,
    test_size=0.2,
    random_state=42,
    n_trials=10,  # Reduzido para economizar tempo
    verbose=True
)

# Personalizar a configuração
config.customize(
    model_types=model_types,
    temperatures=temperatures,
    alphas=alphas
)

# Criar o AutoDistiller
distiller = AutoDistiller(
    dataset=dataset,
    output_dir=output_dir,
    random_state=42,
    verbose=True
)

# Personalizar a configuração
distiller.customize_config(
    model_types=model_types,
    temperatures=temperatures,
    alphas=alphas
)

# Executar os experimentos
print("Executando experimentos de destilação com diferentes temperaturas...")
results_df = distiller.run(verbose_output=False)

# Salvar resultados em CSV para análise posterior
results_df.to_csv(os.path.join(output_dir, "temperatura_resultados.csv"), index=False)
print(f"Experimentos concluídos. Resultados salvos em {output_dir}/temperatura_resultados.csv")